
# Db2でシェアサイクルポートの地理情報分析をやってみよう!
## 地理情報分析

### 前提
 - [地理空間分析機能を有効にしている](https://qiita.com/nishikyon/items/4cd0430e4ebb7d541a82)
 - [JGD2011座標系の設定済み](https://qiita.com/nishikyon/items/fc97391f02b4e9e4bb9c)
 - [データセットアップ のnotebook](https://github.com/kyokonishito/Db2_Spatial_Analysis/blob/main/notebooks/Data_Setup.ipynb)を実行済み
 
  2022/09/28のDataBase Dojoの環境で使用している場合は上記は既に設定済みです
  
 ### 確認済み動作環境
- Watson Studio SaaS版


## 1. 前準備
### 1.1 Db2接続情報のセット

XXXXと9999は自分の接続先のDb2の情報を入れてください

In [ ]:
user = "xxxx" # db2ユーザーid
password = "xxxx" #db2パスワードを入れる
databese = "xxxx" # db2 Db名
host =  "xxxx" # db2 ホスト名
port =  9999 #Db2ポート番号


### 1.2 Db2 Jupyter Notebook Extensions のロード
https://github.com/IBM/db2-jupyter

notebookでdb2 SQLを使いやすくる`Db2 Jupyter Notebook Extensions`をダウンロード&ロードします。

In [ ]:
import os
path = "./db2.ipynb"
if not os.path.exists(path):
    !wget https://raw.githubusercontent.com/IBM/db2-jupyter/master/db2.ipynb
else:
    print("db2.ipynb is existing.")

In [ ]:
%run db2.ipynb

### 1.3 Db2に接続します
`Db2 Jupyter Notebook Extensions`を使用してDb2に接続

In [ ]:
%sql CONNECT TO {databese} USER {user} USING {password} HOST {host} port {port} SSL TRUE

### 1.4 地図表示など必要なライブラリの導入とインポート
以下を使用します。
 - [pandas](https://pandas.pydata.org/) 
 - [geopandas](https://geopandas.org/en/stable/)
 - [folium](http://python-visualization.github.io/folium/)

In [ ]:
# ライブラリーのインストール
!pip install geopandas
!pip install folium

In [ ]:
# ライブラリーのインポート
import pandas as pd
import geopandas as gpd
import folium

- - -

### 2.　都内IBM事業所から1km以内のシェアサイクルポートの情報

#### 2.1 日本全国のシェアサイクルポートの情報
使用するテーブル1:
- SHAREBIKE_STATION 日本全国のシェアサイクルポートの情報

どんな情報?
```sql
SELECT STATION_ID, REGION_ID, CAPACITY, NAME, ADDRESS, URL, COMPANY, 
ST_X(GEO_POINT) LON, ST_Y(GEO_POINT) LAT FROM SHAREBIKE_STATION
```

- [ST_X 関数](https://www.ibm.com/docs/ja/db2/11.5?topic=sf-st-x)
- [ST_Y 関数](https://www.ibm.com/docs/ja/db2/11.5?topic=sf-st-y)

In [ ]:
# ちょっと中身をみます
df = %sql \
    SELECT STATION_ID, REGION_ID, CAPACITY, NAME, ADDRESS, URL, COMPANY, \
    ST_X(GEO_POINT) LON, ST_Y(GEO_POINT) LAT FROM SHAREBIKE_STATION
df

In [ ]:
#地図にプロットしてみます
from folium.plugins import FastMarkerCluster

# 地図生成
folium_map = folium.Map(location=[35, 135], zoom_start=5, height = 500)

# マーカープロット　
FastMarkerCluster(df[['LAT', 'LON']].values.tolist()).add_to(folium_map)

folium_map

#### DBeaver用 

接続設定の方法はこちら：　Qiita 「[DBeaverからDb2 on Cloudにアクセスする](https://qiita.com/nishikyon/items/d0b0e57b72e0996c331c)」

- 設定したFetchサイズ分のみ表示されます(初期設定200)
- WKT形式にしてVARCHARにCASTするとDBeaverで表示可能です
    - [ST_AsText 関数](https://www.ibm.com/docs/ja/db2/11.5?topic=sf-st-astext)
    - > **CAST**( **ST_AsText**(GEO_POINT) AS **VARCHAR(50)** ) AS LOC
    
    
    
- 空間情報列のView/Format→Set "xxx" formatを`Geomerory`にセットしてください

<img src="https://github.com/kyokonishito/Db2_Spatial_Analysis/blob/main/images/dbeaver.jpg?raw=true" width="800px" >



```sql
SELECT STATION_ID, REGION_ID, CAPACITY, NAME, ADDRESS, URL, COMPANY,
CAST( ST_AsText(GEO_POINT)AS VARCHAR(50) ) AS LOC
FROM SHAREBIKE_STATION;
```

- - - 

#### 2.2 IBM事業所情報
使用するテーブル2:
- IBM_LOC 東京都のIBM事業所のテーブル

どんな情報?
```sql
SELECT CODE, BRANCH_NAME, ADDRESS, 
ST_X(GEO_POINT) LON, ST_Y(GEO_POINT) LAT FROM IBM_LOC
```

In [ ]:
# ちょっと中身をみます
df = %sql \
    SELECT CODE, BRANCH_NAME, ADDRESS, \
    ST_X(GEO_POINT) LON, ST_Y(GEO_POINT) LAT FROM IBM_LOC
df

In [ ]:
# 地図生成
folium_map = folium.Map(location=[35.6769883, 139.7588499], zoom_start=11, height = 500)

# マーカープロット　
for i, row in df.iterrows():
    folium.Marker(
        location=[row['LAT'], row['LON']],
        tooltip=row['BRANCH_NAME']
    ).add_to(folium_map)

folium_map

#### DBeaver用 
```sql
SELECT CODE, BRANCH_NAME, ADDRESS, 
CAST( ST_AsText(GEO_POINT)AS VARCHAR(50) ) AS LOC FROM IBM_LOC;
```

- - - 

#### 2.3　登録済みIBM事業所から1km以内のシェアサイクルポートの情報

[ST_Distance関数](https://www.ibm.com/docs/ja/db2/11.5?topic=sf-st-distance)
> **ST_Distance**(IBM.GEO_POINT, ST.GEO_POINT, **'KILOMETER'**) <=1

で1km以内の条件となる

```sql
SELECT ST.NAME, ST.CAPACITY, IBM.BRANCH_NAME, 
       ST_X(ST.GEO_POINT) LON, ST_Y(ST.GEO_POINT) LAT
FROM SHAREBIKE_STATION ST 
CROSS JOIN IBM_LOC IBM 
WHERE ST_Distance(IBM.GEO_POINT, ST.GEO_POINT, 'KILOMETER') <=1
```

In [ ]:
df =%sql \
    SELECT ST.NAME, ST.CAPACITY, IBM.BRANCH_NAME, \
           ST_X(ST.GEO_POINT) LON, ST_Y(ST.GEO_POINT) LAT \
    FROM SHAREBIKE_STATION ST \
    CROSS JOIN IBM_LOC IBM \
    WHERE ST_Distance(IBM.GEO_POINT ,  ST.GEO_POINT, 'KILOMETER') <=1
df 

In [ ]:
# 地図生成
folium_map = folium.Map(location=[35.6769883, 139.7588499], zoom_start=12, height = 500)

# マーカープロット　
for i, row in df.iterrows():
    folium.CircleMarker(
        location=[row['LAT'], row['LON']],
        tooltip=row['NAME'],
        radius=5,
        color='green',
        fill_color='green'
    ).add_to(folium_map)

#IBM事業所から1km プロット
df_ibm_loc = %sql \
    SELECT BRANCH_NAME, ADDRESS, ST_X(GEO_POINT) LON, ST_Y(GEO_POINT) LAT \
    FROM IBM_LOC 
for i, row in df_ibm_loc.iterrows():
    folium.Circle(
        [row['LAT'], row['LON']],
        radius=1000,
    ).add_to(folium_map)
    folium.Marker(
        location=[row['LAT'], row['LON']],
        tooltip=row['BRANCH_NAME']
    ).add_to(folium_map)

folium_map

#### DBeaver用 
```sql
SELECT ST.NAME, ST.CAPACITY, CAST( ST_AsText(ST.GEO_POINT)AS VARCHAR(50) ) AS LOC, 
IBM.BRANCH_NAME, CAST( ST_AsText(IBM.GEO_POINT)AS VARCHAR(50) )AS IBM_LOC
FROM SHAREBIKE_STATION ST 
CROSS JOIN IBM_LOC IBM 
WHERE ST_Distance(IBM.GEO_POINT, ST.GEO_POINT, 'KILOMETER') <=1 ;
```

- - - 

### 3.千代田区と中央区と港区のシェアサイクルポートの情報
使用するテーブル1:
- SHAREBIKE_STATION 日本全国のシェアサイクルポートの情報

使用するテーブル2:
- CITY_SHAPE 東京都市区町村境界

### 3.1 東京都市区町村境界
どんな情報?
```sql
SELECT SE_ROW_ID ,N03_001 ,N03_002 ,N03_003, N03_004, N03_007, 
CAST(ST_AsText(GEO) AS VARCHAR(30000)) LOC  
FROM CITY_SHAPE WHERE N03_004 IN ('中央区', '千代田区', '港区')
```

In [ ]:
df_area =%sql \
    SELECT OBJECTID ,N03_001 ,N03_002 ,N03_003, N03_004, N03_007,\
    CAST(ST_AsText(GEO) AS VARCHAR(30000)) LOC  \
    FROM CITY_SHAPE WHERE N03_004 IN ('中央区', '千代田区', '港区')
display(df_area)

In [ ]:
# 地図生成
folium_map = folium.Map(location=[35.6636391,139.7606268], zoom_start=12, height = 500)

gdf = gpd.GeoDataFrame(df_area, geometry = gpd.GeoSeries.from_wkt(df_area.LOC))
geo_j = gdf['geometry'].to_json()
folium.GeoJson(geo_j, style_function=lambda x: {'fillColor': 'orange'}).add_to(folium_map)
folium_map

#### DBeaver用 (実行は省略します)
```sql
SELECT OBJECTID ,N03_001 ,N03_002 ,N03_003, N03_004, N03_007, 
CAST(ST_AsText(GEO) AS VARCHAR(30000)) LOC  
FROM CITY_SHAPE WHERE N03_004 IN ('中央区', '千代田区', '港区')；
```

- - - 


### 3.2 千代田区と中央区と港区のシェアサイクルポートの情報
**[ST_Within（形状1, 形状2)](https://www.ibm.com/docs/ja/db2/11.5?topic=sf-st-within)**: 形状2が形状1を内包するか
> **ST_Within**(ST.GEO_POINT, AREA.GEO)

```sql
with AREA AS (
    SELECT GEO
    FROM CITY_SHAPE 
    WHERE N03_004 IN ('中央区', '千代田区', '港区')
)
SELECT ST.NAME, ST.CAPACITY, ST_X(ST.GEO_POINT) LON, ST_Y(ST.GEO_POINT) LAT
FROM SHAREBIKE_STATION ST, AREA
WHERE ST_Within(ST.GEO_POINT, AREA.GEO)
```

In [ ]:
df = %sql \
    with AREA AS ( \
    SELECT GEO \
    FROM CITY_SHAPE  \
    WHERE N03_004 IN ('中央区', '千代田区', '港区')\
    )\
    SELECT ST.NAME, ST.CAPACITY, ST_X(ST.GEO_POINT) LON, ST_Y(ST.GEO_POINT) LAT \
    FROM SHAREBIKE_STATION ST, AREA \
    WHERE ST_Within(ST.GEO_POINT, AREA.GEO)
display(df)


In [ ]:
#マーカープロット＋地図表示

for i, row in df.iterrows():
    folium.CircleMarker(
        location=[row['LAT'], row['LON']],
        tooltip=row['NAME'],
        radius=5,
        color='green',
        fill_color='green'
    ).add_to(folium_map)


folium_map

#### DBeaver用 (実行は省略します)
```sql
with AREA AS (
    SELECT GEO
    FROM CITY_SHAPE 
    WHERE N03_004 IN ('中央区', '千代田区', '港区')
)
SELECT ST.NAME, ST.CAPACITY, CAST( ST_AsText(ST.GEO_POINT)AS VARCHAR(50) ) AS LOC, 
AREA.N03_004, CAST( ST_AsText(AREA.GEO)AS VARCHAR(30000) )AS AREA
FROM SHAREBIKE_STATION ST, AREA
WHERE ST_Within(ST.GEO_POINT, AREA.GEO) ;
```

- - -

### 4.千代田区と中央区と港区のシェアサイクルポートの売上分析
使用するテーブル1:
- SHAREBIKE_STATION 日本全国のシェアサイクルポートの情報

使用するテーブル2:
- STATION_SALES シェアサイクルポート月別売上情報




### 4.1 シェアサイクルポート月別売上情報
どんな情報?
```sql
SELECT *FROM STATION_SALES FETCH FIRST 10 ROWS ONLY
```

In [ ]:
%sql SELECT *FROM STATION_SALES FETCH FIRST 10 ROWS ONLY


### 4.2 千代田区と中央区と港区のシェアサイクルポートの売上TOP100
**[ST_Within（形状1, 形状2)](https://www.ibm.com/docs/ja/db2/11.5?topic=sf-st-within)**: 形状2が形状1を内包するか
```sql
WITH TOKYO3_ST AS ( 
        WITH SHAPE  AS ( SELECT GEO FROM CITY_SHAPE WHERE N03_004 IN ('中央区', '千代田区', '港区') )
        SELECT STATION_ID, NAME, ST_X(GEO_POINT) LON, ST_Y(GEO_POINT) LAT
        FROM SHAREBIKE_STATION , SHAPE
        WHERE ST_Within (GEO_POINT, GEO)
    ),
    ANUAL_SALES AS (
        SELECT STATION_ID,SUM(AMOUNT) AS AMOUNT
        FROM STATION_SALES 
        GROUP BY  STATION_ID 
        ORDER BY AMOUNT DESC 
    ) 
    SELECT T.STATION_ID, NAME, LOC, LON, LAT, AMOUNT 
    FROM TOKYO3_ST T, ANUAL_SALES S 
    WHERE T.STATION_ID = S.STATION_ID 
    ORDER BY AMOUNT DESC LIMIT 100
```


In [ ]:
df = %sql \
    WITH TOKYO3_ST AS (\
        WITH SHAPE AS (SELECT GEO FROM CITY_SHAPE WHERE N03_004 IN ('中央区', '千代田区', '港区') ) \
        SELECT STATION_ID, NAME, ST_X(GEO_POINT) LON, ST_Y(GEO_POINT) LAT\
        FROM SHAREBIKE_STATION , SHAPE \
        WHERE ST_Within (GEO_POINT, GEO) \
    ), \
    ANUAL_SALES AS ( \
        SELECT STATION_ID,SUM(AMOUNT) AS AMOUNT \
        FROM STATION_SALES \
        GROUP BY  STATION_ID \
        ORDER BY AMOUNT DESC \
    ) \
    SELECT T.STATION_ID, NAME, LON, LAT, AMOUNT \
    FROM TOKYO3_ST T, ANUAL_SALES S \
    WHERE T.STATION_ID = S.STATION_ID \
    ORDER BY AMOUNT DESC LIMIT 100
df

In [ ]:
# 地図表示
df_area =%sql SELECT CAST(ST_AsText(GEO) AS VARCHAR(30000)) LOC FROM CITY_SHAPE WHERE N03_004 IN ('中央区', '千代田区', '港区') 

# 地図生成3

folium_map = folium.Map(location=[35.6636391,139.7606268], zoom_start=12, height = 500)

gdf = gpd.GeoDataFrame(df_area, geometry = gpd.GeoSeries.from_wkt(df_area.LOC))
geo_j = gdf['geometry'].to_json()
folium.GeoJson(geo_j, style_function=lambda x: {'fillColor': 'orange'}).add_to(folium_map)

# マーカープロット　
for i, row in df.iterrows():
    folium.CircleMarker(
        location=[row['LAT'], row['LON']],
        tooltip=row['NAME']+'<br>'+'年間売上 ¥'+format(row['AMOUNT'], ',.0f'),
        radius=5,
        color='green',
        fill_color='green'
    ).add_to(folium_map)


folium_map

### DBeaver用 (実行は省略します)
```sql
WITH TOKYO3_ST AS ( 
        WITH SHAPE  as (SELECT GEO FROM CITY_SHAPE WHERE N03_004 IN ('中央区', '千代田区', '港区') )
        SELECT STATION_ID, NAME, ST_AsText( GEO_POINT) AS LOC, ST_X(GEO_POINT) LON ,
        ST_Y(GEO_POINT) LAT, ST_AsText(GEO) AS GEO 
        FROM SHAREBIKE_STATION , SHAPE
        WHERE ST_Within (GEO_POINT, GEO)
    ),
    ANUAL_SALES AS (
        SELECT STATION_ID,SUM(AMOUNT) AS AMOUNT
        FROM STATION_SALES 
        GROUP BY  STATION_ID 
        ORDER BY AMOUNT DESC 
    ) 
    SELECT T.STATION_ID, NAME, CAST(LOC AS VARCHAR(50)) AS LOC, LON, LAT, CAST(GEO AS VARCHAR(10000)) AS GEO, AMOUNT 
    FROM TOKYO3_ST T, ANUAL_SALES S 
    WHERE T.STATION_ID = S.STATION_ID 
    ORDER BY AMOUNT DESC LIMIT 100
```

- - -

### 4.3 千代田区と中央区と港区のシェアサイクルポートの売上Heatmap
**[ST_Within（形状1, 形状2)](https://www.ibm.com/docs/ja/db2/11.5?topic=sf-st-within)**: 形状2が形状1を内包するか
```sql
WITH TOKYO3_ST AS ( 
        WITH SHAPE  AS ((SELECT GEO FROM CITY_SHAPE WHERE N03_004 IN ('中央区', '千代田区', '港区'))
        SELECT STATION_ID, NAME, ST_X(GEO_POINT) LON, ST_Y(GEO_POINT) LAT
        FROM SHAREBIKE_STATION , SHAPE
        WHERE ST_Within (GEO_POINT, GEO)
    ),
    ANUAL_SALES AS (
        SELECT STATION_ID,SUM(AMOUNT) AS AMOUNT
        FROM STATION_SALES 
        GROUP BY  STATION_ID 
        ORDER BY AMOUNT DESC 
    ) 
    SELECT T.STATION_ID, NAME, LOC, LON, LAT, AMOUNT 
    FROM TOKYO3_ST T, ANUAL_SALES S 
    WHERE T.STATION_ID = S.STATION_ID 
    ORDER BY AMOUNT DESC
```

In [ ]:
 # 千代田区と中央区と港区のシェアサイクルポートの売上一覧
df = %sql \
        WITH TOKYO3_ST AS ( \
        WITH SHAPE  AS (SELECT GEO FROM CITY_SHAPE WHERE N03_004 IN ('中央区', '千代田区', '港区') )\
        SELECT STATION_ID, NAME, ST_X(GEO_POINT) LON, ST_Y(GEO_POINT) LAT\
        FROM SHAREBIKE_STATION , SHAPE \
        WHERE ST_Within (GEO_POINT, GEO) \
    ), \
    ANUAL_SALES AS ( \
        SELECT STATION_ID,SUM(AMOUNT) AS AMOUNT \
        FROM STATION_SALES \
        GROUP BY  STATION_ID \
        ORDER BY AMOUNT DESC \
    ) \
    SELECT T.STATION_ID, NAME, LON, LAT, AMOUNT \
    FROM TOKYO3_ST T, ANUAL_SALES S \
    WHERE T.STATION_ID = S.STATION_ID \
    ORDER BY AMOUNT DESC
#     LIMIT 100

df

In [ ]:
#千代田区と中央区と港区のシェアサイクルポートの売上Heatmap
# 地図表示
df_area =%sql SELECT CAST(ST_AsText(GEO) AS VARCHAR(30000)) LOC FROM CITY_SHAPE WHERE N03_004 IN ('中央区', '千代田区', '港区') 

# 地図生成　
folium_map = folium.Map(location=[35.6636391,139.7606268], zoom_start=12, height = 500)


#3つの区の境界
gdf = gpd.GeoDataFrame(df_area, geometry = gpd.GeoSeries.from_wkt(df_area.LOC))
geo_j = gdf['geometry'].to_json()
folium.GeoJson(geo_j, style_function=lambda x: {'fillColor': '#00000000'}).add_to(folium_map)

#Heatmap作成
heat_data_df = df.iloc[:, 2:5][['LAT', 'LON', 'AMOUNT']]
folium.plugins.HeatMap(
    data = heat_data_df.values, # ２次元を渡す
    radius=10,
    blur=5
).add_to(folium_map)


#サイクルポートのマーカー
for i, row in df.iterrows():
    folium.CircleMarker(
        location=[row['LAT'], row['LON']],
        tooltip=row['NAME']+'<br>'+'年間売上 ¥'+format(row['AMOUNT'], ',.0f'),
        radius=1,
        color='green',
        fill_color='green'
    ).add_to(folium_map)


In [ ]:
#地図の表示
folium_map

In [ ]:
###  DB接続の切断
%sql connect close

以上です。お疲れ様でした!